### Par: HOUSNI Kamar

## Projet : Metaheuristic

### Constructive heuristic- Finite Best Strip

* We will start by reading our data;
    * we extract the bin and item dimensions in order to perform our heuristic algorithm on the data.

In [1]:
def read_dataset(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()
    # first we extract the bin dimensions
    width_bins = int(lines[1].split(':')[1].strip())
    height_bins = int(lines[2].split(':')[1].strip())
    
    # then we extract the item dimensions
    items = []
    for line in line[4:]:
        if line.strip() == '':
            continue
        item_width = int(line.split(':')[1].strip())
        item_height = int(line.split(':')[2].strip())
        items.append((item_width, item_height))
        
        return width_bins, height_bins, items

In [2]:
def fbs_heuristic(bin_width, bin_height, items):
    # Sort items by non-increasing height
    items.sort(key=lambda x: x[1], reverse=True)

    # Phase 1: Pack items into an infinite height strip
    strip_width = bin_width
    shelves = []
    for item in items:
        shelf_height = item[1]
        # Find the best fit shelf for the item
        best_shelf = None
        best_residual_width = float('inf')
        for shelf in shelves:
            if shelf[0] >= item[0] and shelf[1] < best_residual_width:
                best_shelf = shelf
                best_residual_width = shelf[1]
        if best_shelf is None:
            # Create a new shelf for the item
            shelves.append((item[0], strip_width - item[0]))
        else:
            # Pack the item onto the best fit shelf
            shelves.remove(best_shelf)
            shelves.append((best_shelf[0], best_shelf[1] - item[0]))

    # Phase 2: Pack shelves into finite bins
    bins = []
    for shelf in shelves:
        # Find the best fit bin for the shelf
        best_bin = None
        best_residual_height = float('inf')
        for bin in bins:
            if bin[1] >= shelf[0] and bin[0] < best_residual_height:
                best_bin = bin
                best_residual_height = bin[0]
        if best_bin is None:
            # Create a new bin for the shelf
            bins.append((bin_height - shelf[0], strip_width))
        else:
            # Pack the shelf into the best fit bin
            bins.remove(best_bin)
            bins.append((best_bin[0] - shelf[0], best_bin[1]))

    return len(bins)

In [3]:
bin_width, bin_height, items = read_dataset('instance2dBIN.txt')
print(f"Bin dimensions: {bin_width} x {bin_height}")
print("Item dimensions:")
for item in items:
    print(f"{item[0]} x {item[1]}")

num_bins = fbs_heuristic(bin_width, bin_height, items)
print(f"Number of bins used: {num_bins}")


UnboundLocalError: cannot access local variable 'line' where it is not associated with a value